<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Start-to-Finish Example: Unit Testing `GiRaFFE_NRPy`: Induction Equation

## Author: Patrick Nelson
### Formatting improvements courtesy Brandon Clark

## This module validates the routine to compute the flux of $\epsilon_{ijk} v^j B^k$ for `GiRaFFE`.

**Notebook Status:** <font color='red'><b>In-Progress</b></font>

**Validation Notes:** This module will validate the routines in [Tutorial-GiRaFFE_NRPy-Induction_Equation](Tutorial-GiRaFFE_NRPy-Induction_Equation.ipynb).

### NRPy+ Source Code for this module: 
* [GiRaFFE_NRPy/Induction_Equation.py](../../edit/in_progress/GiRaFFE_NRPy/Induction_Equation.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-Induction_Equation.ipynb) Generates the driver to compute the magnetic field from the vector potential in arbitrary spactimes.

## Introduction:

This notebook validates our algorithm to compute the flux of $\epsilon_{ijk} v^j B^k$ through cell faces, which contributes to the right-hand side of the evolution equation for $A_i$, for use in `GiRaFFE_NRPy`. Because the original `GiRaFFE` used staggered grids and we do not, we can not trivially do a direct comparison to the old code. Instead, we will compare the numerical results with the expected analytic results. 

It is, in general, good coding practice to unit test functions individually to verify that they produce the expected and intended output. Here, we expect our functions to produce the correct cross product between the velocity and magnetic field in an arbitrary spacetime. To that end, we will choose functions with relatively simple analytic forms.

In this test, we will generate analytic forms for the magnetic field and three-velocity. We will need to do this in a $7 \times 7 \times 7$ cube in order to run the PPM routine on the data to generate the inputs compute the non-gauge terms of the RHS of the induction equation, unless there's a way to usefully spoof the left- and right-face values for the HLLE solver. We care about this here, because we are comparing against an analytic expression and not the old code.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#setup): Set up core functions and parameters for unit testing the A2B algorithm
    1. [Step 1.a](#magnetic) Set analytic magnetic field
    1. [Step 1.b](#velocity) Set analytic Valencia three-velocity
    1. [Step 1.c](#free_parameters) Set free parameters in the code
1. [Step 2](#mainc): `Induction_Equation_unit_test.c`: The Main C Code
    1. [Step 2.a](#compile_run): Compile and run the code
1. [Step 3](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='setup'></a>

# Step 1: Set up core functions and parameters for unit testing the A2B algorithm \[Back to [top](#toc)\]

$$\label{setup}$$

We'll start by appending the relevant paths to `sys.path` so that we can access sympy modules in other places. Then, we'll import NRPy+ core functionality and set up a directory in which to carry out our test. We must also set the desired finite differencing order.

In [1]:
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions
# First, we'll add the parent directory to the list of directories Python will check for modules.
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import *            # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

out_dir = "Validation/"
cmd.mkdir(out_dir)

thismodule = "Start_to_Finish-GiRaFFE_NRPy-Induction_Equation"
Use_Shock_Data = False


<a id='velocity'></a>

## Step 1.a: Valencia three-velocity \[Back to [top](#toc)\]
$$\label{velocity}$$

Here, we'll generate some functions for the velocity. Let's choose arctangents, since those have asymptotes that can be easily manipulated to prevent accidentally setting superluminal speeds. 
\begin{align}
\bar{v}^x &= \frac{2}{\pi} \arctan(ax + by + cz) \\
\bar{v}^y &= \frac{2}{\pi} \arctan(bx + cy + az) \\
\bar{v}^z &= \frac{2}{\pi} \arctan(cx + ay + bz) \\
\end{align}
If we want to add a jump at the origin, we can simply add $\max(0,x)$ to the argument of the arctangent. This will add a shock in the $x$-direction. The maximum will be described without the use of if statements as 
$$
\max(a,b) = \tfrac{1}{2} \left( a+b + \lvert a-b \rvert \right). 
$$

In [2]:
def max_noif(a,b):
    return sp.Rational(1,2)*(a+b+nrpyAbs(a-b))

a,b,c = par.Cparameters("REAL",thismodule,["a","b","c"],1e300) # Note that this default value allows us to set
                                                               # these directly in the C code
M_PI  = par.Cparameters("#define",thismodule,["M_PI"], "")

par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()
x = rfm.xxCart[0]
y = rfm.xxCart[1]
z = rfm.xxCart[2]

args = ixp.zerorank1()
args[0] = a*x + b*y + c*z
args[1] = b*x + c*y + a*z
args[2] = c*x + a*y + b*z
if Use_Shock_Data: 
    for i in range(3): 
        args[i] += max_noif(0,x)

ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","ValenciavU")
for i in range(3): 
    ValenciavU[i] = (sp.sympify(2.0)/M_PI)*sp.atan(args[i])

<a id='magnetic'></a>

## Step 1.b: Magnetic field \[Back to [top](#toc)\]
$$\label{magnetic}$$

We'll also need some functions for the magnetic field. Exponentials sound fun.
\begin{align}
B^x &= \exp(ey+fz) \\
B^y &= \exp(fz+dx) \\
B^z &= \exp(dx+ey) \\
\end{align}
In this case, we'll add $\max{0,x}$ to the field to add the jump.

In [3]:
d,e,f = par.Cparameters("REAL",thismodule,["d","e","f"],1e300) # Note that this default value allows us to set
                                                               # these directly in the C code
BU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","BU")
BU[0] = sp.exp(e*y+f*z)
BU[1] = sp.exp(f*z+d*x)
BU[2] = sp.exp(d*x+e*y)
if Use_Shock_Data: 
    for i in range(3): 
        BU[i] += max_noif(0,x)


<a id='functions'></a>

## Step 1.c: Generate C functions to write the test data \[Back to [top](#toc)\]
$$\label{functions}$$


In [4]:
BU_to_print = [\
                lhrh(lhs=gri.gfaccess("out_gfs","BU0"),rhs=BU[0]),\
                lhrh(lhs=gri.gfaccess("out_gfs","BU1"),rhs=BU[1]),\
                lhrh(lhs=gri.gfaccess("out_gfs","BU2"),rhs=BU[2]),\
               ]

desc = "Calculate sample magnetic field data"
name = "calculate_BU"
outCfunction(
    outfile  = os.path.join(out_dir,name+".h"), desc=desc, name=name,
    params   ="const paramstruct *params,REAL *xx[3],REAL *auxevol_gfs",
    body     = fin.FD_outputC("returnstring",BU_to_print,params="outCverbose=False").replace("IDX4","IDX4S"),
    loopopts="AllPoints,Read_xxs")

ValenciavU_to_print = [\
                       lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU0"),rhs=ValenciavU[0]),\
                       lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU1"),rhs=ValenciavU[1]),\
                       lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU2"),rhs=ValenciavU[2]),\
                      ]

desc = "Calculate sample velocity data"
name = "calculate_ValenciavU"
outCfunction(
    outfile  = os.path.join(out_dir,name+".h"), desc=desc, name=name,
    params   ="const paramstruct *params,REAL *xx[3],REAL *auxevol_gfs",
    body     = fin.FD_outputC("returnstring",ValenciavU_to_print,params="outCverbose=False").replace("IDX4","IDX4S"),
    loopopts="AllPoints,Read_xxs")


<a id='free_parameters'></a>

## Step 1.d: Set free parameters in the code \[Back to [top](#toc)\]
$$\label{free_parameters}$$

We also need to create the files that interact with NRPy's C parameter interface. 

In [ ]:
# Step 3.d.i: Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h

# Step 3.d.ii: Set free_parameters.h
with open(os.path.join(out_dir,"free_parameters.h"),"w") as file:
    file.write("""
// Override parameter defaults with values based on command line arguments and NGHOSTS.
// We'll use this grid. It has one point and one ghost zone.
const int NGHOSTS = 3;
params.Nxx0 = 1;
params.Nxx1 = 1;
params.Nxx2 = 1;
params.Nxx_plus_2NGHOSTS0 = params.Nxx0 + 2*NGHOSTS;
params.Nxx_plus_2NGHOSTS1 = params.Nxx1 + 2*NGHOSTS;
params.Nxx_plus_2NGHOSTS2 = params.Nxx2 + 2*NGHOSTS;
// Step 0d: Set up space and time coordinates
// Step 0d.i: Declare \Delta x^i=dxx{0,1,2} and invdxx{0,1,2}, as well as xxmin[3] and xxmax[3]:
const REAL xxmin[3] = {-1.0,-1.0,-1.0};
const REAL xxmax[3] = { 1.0, 1.0, 1.0};

params.dxx0 = (xxmax[0] - xxmin[0]) / ((REAL)params.Nxx_plus_2NGHOSTS0-1.0);
params.dxx1 = (xxmax[1] - xxmin[1]) / ((REAL)params.Nxx_plus_2NGHOSTS1-1.0);
params.dxx2 = (xxmax[2] - xxmin[2]) / ((REAL)params.Nxx_plus_2NGHOSTS2-1.0);
//printf("dxx0,dxx1,dxx2 = %.5e,%.5e,%.5e\\n",params.dxx0,params.dxx1,params.dxx2);
params.invdx0 = 1.0 / params.dxx0;
params.invdx1 = 1.0 / params.dxx1;
params.invdx2 = 1.0 / params.dxx2;
\n""")

# Generates declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(out_dir))

<a id='module'></a>

## Step 1.e: Generate `GiRaFFE_NRPy` Files \[Back to [top](#toc)\]
$$\label{module}$$

Here, we generate the functions we want to test by calling the function found [here](../../edit/in_progress/GiRaFFE_NRPy/GiRaFFE_NRPy_PPM.py) and documented in [this tutorial](Tutorial-Start_to_Finish-GiRaFFE_NRPy-PPM.ipynb).

In [ ]:
import GiRaFFE_NRPy.GiRaFFE_NRPy_PPM as PPM
PPM.GiRaFFE_NRPy_PPM(out_dir)

<a id='electric_flux'></a>

## Step 1.f: Calculate the flux of $\epsilon_{ijk} v^j B^k$ \[Back to [top](#toc)\]
$$\label{electric_flux}$$

Here, we generate the functions necessary to calculate the electric flux on the cell faces, which is the algorithm we are specifically trying to test. 

In [ ]:
import GiRaFFE_NRPy.Induction_Equation as Ef

# We will pass values of the gridfunction on the cell faces into the function. This requires us
# to declare them as C parameters in NRPy+. We will denote this with the _face infix/suffix.
alpha_face = gri.register_gridfunctions("AUXEVOL","alpha_face")
gamma_faceDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gamma_faceDD","sym01")
beta_faceU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","beta_faceU")

# We'll need some more gridfunctions, now, to represent the reconstructions of BU and ValenciavU
# on the right and left faces
Valenciav_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","Valenciav_rU",DIM=3)
B_rU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_rU",DIM=3)
Valenciav_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","Valenciav_lU",DIM=3)
B_lU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_lU",DIM=3)

Ef.calculate_E_i_flux(True,alpha_face,gamma_faceDD,beta_faceU,\
                      Valenciav_rU,B_rU,Valenciav_lU,B_lU)

for flux_dirn in range(3):
    E_i_flux_to_print = [\
                         lhrh(lhs=gri.gfaccess("out_gfs","E_fluxD0"),rhs=E_fluxD[0]),\
                         lhrh(lhs=gri.gfaccess("out_gfs","E_fluxD1"),rhs=E_fluxD[1]),\
                         lhrh(lhs=gri.gfaccess("out_gfs","E_fluxD2"),rhs=E_fluxD[2]),\
                        ]

    desc = "Calculate the electric flux in direction " + str(flux_dirn) + "."
    name = "calculate_E_fluxD" + str(flux_dirn)
    outCfunction(
        outfile  = os.path.join(out_dir,name+".h"), desc=desc, name=name,
        params   ="const paramstruct *params,REAL *xx[3],REAL *auxevol_gfs",
        body     = fin.FD_outputC("returnstring",E_i_flux_to_print,params="outCverbose=False").replace("IDX4","IDX4S"),
        loopopts ="InteriorPoints,Read_xxs")


<a id='exact_flux'></a>

## Step 1.g: Calculate the *exact* flux of $\epsilon_{ijk} v^j B^k$ \[Back to [top](#toc)\]
$$\label{exact_flux}$$

Here, we generate a function to analytically calculate the electric flux on the cell faces for comparison. We'll need to import the Levi-Civita tensor for this.

In [ ]:
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
# We'll re-enable this once we move into curved spacetimes.
# for i in range(3):
#     for j in range(3):
#         for k in range(3):
#             LeviCivitaDDD[i][j][k] = LeviCivitaDDD[i][j][k] / sp.sqrt(gammadet)

AD = ixp.zerorank1()
for i in range(3):
    for j in range(3):
        for k in range(3):
            AD[i] = LeviCivitaDDD[i][j][k]*driftvU[j]*BU[k] # TODO: Find best way to get driftvU here. 


<a id='mainc'></a>

# Step 2: `Induction_Equation_unit_test.c`: The Main C Code \[Back to [top](#toc)\]
$$\label{mainc}$$

Now that we have our vector potential and analytic magnetic field to compare against, we will start writing our unit test. We'll also import common C functionality, define `REAL`, the number of ghost zones, and the faces, and set the standard macros for NRPy+ style memory access.

In [8]:
%%writefile $out_dir/Induction_Equation_unit_test.c
// These are common packages that we are likely to need.
#include "stdio.h"
#include "stdlib.h"
#include "math.h"
#include "string.h" // Needed for strncmp, etc.
#include "stdint.h" // Needed for Windows GCC 6.x compatibility
#include <time.h>   // Needed to set a random seed.

#define REAL double
#include "declare_Cparameters_struct.h"

REAL a,b,c,d,e,f;

// Standard NRPy+ memory access:
#define IDX4S(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) + Nxx_plus_2NGHOSTS2 * (g) ) ) )

#define LOOP_REGION(i0min,i0max, i1min,i1max, i2min,i2max) \
  for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++)


Overwriting Validation//A2B_unit_test.c


We'll now define the gridfunction names.

In [9]:
%%writefile -a $out_dir/Induction_Equation_unit_test.c
// Let's also #define the NRPy+ gridfunctions
#define VALENCIAVU0GF 0
#define VALENCIAVU1GF 1
#define VALENCIAVU2GF 2
#define BU0GF 3
#define BU1GF 4
#define BU2GF 5
#define VALENCIAV_RU0GF 6
#define VALENCIAV_RU1GF 7
#define VALENCIAV_RU2GF 8
#define B_RU0GF 9
#define B_RU1GF 10
#define B_RU2GF 11
#define VALENCIAV_LU0GF 12
#define VALENCIAV_LU1GF 13
#define VALENCIAV_LU2GF 14
#define B_LU0GF 15
#define B_LU1GF 16
#define B_LU2GF 17
#define GAMMA_FACEDD00GF 18
#define GAMMA_FACEDD01GF 19
#define GAMMA_FACEDD02GF 20
#define GAMMA_FACEDD11GF 21
#define GAMMA_FACEDD12GF 22
#define GAMMA_FACEDD22GF 23
#define BETA_FACEU0GF 24
#define BETA_FACEU1GF 25
#define BETA_FACEU2GF 26
#define ALPHA_FACEGF 27
#define NUM_AUXEVOL_GFS 28

#define AD0GF 0
#define AD1GF 1
#define AD2GF 2
#define NUM_EVOL_GFS 3

Appending to Validation//A2B_unit_test.c


Now, we'll handle the different A2B codes. There are several things to do here. First, we'll add `#include`s to the C code so that we have access to the functions we want to test. We must also create a directory and copy the files to that directory. We will choose to do this in the subfolder `A2B` relative to this tutorial.


In [10]:
%%writefile -a $out_dir/Induction_Equation_unit_test.c
// Some specific definitions needed for this file
typedef struct __gf_and_gz_struct__ {
  REAL *gf;
  int gz_lo[4],gz_hi[4];
} gf_and_gz_struct;

const int VX=0,VY=1,VZ=2,BX=3,BY=4,BZ=5;
const int NUM_RECONSTRUCT_GFS = 6;
const int MAXNUMVARS = NUM_RECONSTRUCT_GFS; // For the CCTK version

#include "reconstruct_set_of_prims_PPM_GRFFE_NRPy.c"
#include "loop_defines_reconstruction_NRPy.h"

#include "calculate_BU.h"
#include "calculate_ValenciavU.h"

#include ".h" // This file contains both functions we need.


Appending to Validation//A2B_unit_test.c


Now, we'll write the main method. First, we'll set up the grid. In this test, we cannot use only one point. As we are testing a three-point stencil, we can get away with a minimal $3 \times 3 \times 3$ grid. Then, we'll write the A fields. After that, we'll calculate the magnetic field two ways.

In [11]:
%%writefile -a $out_dir/Induction_Equation_unit_test.c
int main(int argc, const char *argv[]) {
    paramstruct params;
#include "set_Cparameters_default.h"

    // Step 0c: Set free parameters, overwriting Cparameters defaults 
    //          by hand or with command-line input, as desired.
#include "free_parameters.h"
#include "set_Cparameters-nopointer.h"

    // We'll define our grid slightly different from how we normally would. We let our outermost
    // ghostzones coincide with xxmin and xxmax instead of the interior of the grid. This means
    // that the ghostzone points will have identical positions so we can do convergence tests of them.    // Step 0d.ii: Set up uniform coordinate grids
    REAL *xx[3];
    xx[0] = (REAL *)malloc(sizeof(REAL)*Nxx_plus_2NGHOSTS0);
    xx[1] = (REAL *)malloc(sizeof(REAL)*Nxx_plus_2NGHOSTS1);
    xx[2] = (REAL *)malloc(sizeof(REAL)*Nxx_plus_2NGHOSTS2);
    for(int j=0;j<Nxx_plus_2NGHOSTS0;j++) xx[0][j] = xxmin[0] + ((REAL)(j))*dxx0;
    for(int j=0;j<Nxx_plus_2NGHOSTS1;j++) xx[1][j] = xxmin[1] + ((REAL)(j))*dxx1;
    for(int j=0;j<Nxx_plus_2NGHOSTS2;j++) xx[2][j] = xxmin[2] + ((REAL)(j))*dxx2;
    
    for(int j=0;j<Nxx_plus_2NGHOSTS0;j++) printf("x[%d] = %.5e\n",j,xx[0][j]);

    // This is the array to which we'll write the NRPy+ variables.
    REAL *auxevol_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS2 * Nxx_plus_2NGHOSTS1 * Nxx_plus_2NGHOSTS0);
    REAL *rhs_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS2 * Nxx_plus_2NGHOSTS1 * Nxx_plus_2NGHOSTS0);
    REAL *rhs_exact_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS2 * Nxx_plus_2NGHOSTS1 * Nxx_plus_2NGHOSTS0);
    
    LOOP_REGION(0,Nxx_plus_2NGHOSTS2,0,Nxx_plus_2NGHOSTS1,0,Nxx_plus_2NGHOSTS0) {
        auxevol_gfs[IDX4S(GAMMA_FACEDD00GF,i0,i1,i2)] = 1.0; // Flat Space
        auxevol_gfs[IDX4S(GAMMA_FACEDD01GF,i0,i1,i2)] = 0.0; // Flat Space
        auxevol_gfs[IDX4S(GAMMA_FACEDD02GF,i0,i1,i2)] = 0.0; // Flat Space
        auxevol_gfs[IDX4S(GAMMA_FACEDD11GF,i0,i1,i2)] = 1.0; // Flat Space
        auxevol_gfs[IDX4S(GAMMA_FACEDD12GF,i0,i1,i2)] = 0.0; // Flat Space
        auxevol_gfs[IDX4S(GAMMA_FACEDD22GF,i0,i1,i2)] = 1.0; // Flat Space
        auxevol_gfs[IDX4S(BETA_FACEU0GF,i0,i1,i2)] = 0.0; // Flat Space
        auxevol_gfs[IDX4S(BETA_FACEU1GF,i0,i1,i2)] = 0.0; // Flat Space
        auxevol_gfs[IDX4S(BETA_FACEU2GF,i0,i1,i2)] = 0.0; // Flat Space
        auxevol_gfs[IDX4S(ALPHA_FACEGF,i0,i1,i2)] = 1.0; // Flat Space
    }
    
    a = (double)(rand()%10-5);
    b = (double)(rand()%10-5);
    c = (double)(rand()%10-5);
    d = (double)(rand()%10-5);
    e = (double)(rand()%10-5);
    f = (double)(rand()%10-5);

        gf_and_gz_struct in_prims[NUM_RECONSTRUCT_GFS], out_prims_r[NUM_RECONSTRUCT_GFS], out_prims_l[NUM_RECONSTRUCT_GFS];
    int which_prims_to_reconstruct[NUM_RECONSTRUCT_GFS],num_prims_to_reconstruct;

    const int Nxxp2NG012 = Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2;
    REAL temporary[Nxxp2NG012];

    int ww=0;
    in_prims[ww].gf      = auxevol_gfs + Nxxp2NG012*VALENCIAVU0GF; 
      out_prims_r[ww].gf = auxevol_gfs + Nxxp2NG012*VALENCIAV_RU0GF; 
      out_prims_l[ww].gf = auxevol_gfs + Nxxp2NG012*VALENCIAV_LU0GF; 
    ww++;
    in_prims[ww].gf      = auxevol_gfs + Nxxp2NG012*VALENCIAVU1GF; 
      out_prims_r[ww].gf = auxevol_gfs + Nxxp2NG012*VALENCIAV_RU1GF; 
      out_prims_l[ww].gf = auxevol_gfs + Nxxp2NG012*VALENCIAV_LU1GF; 
    ww++;
    in_prims[ww].gf      = auxevol_gfs + Nxxp2NG012*VALENCIAVU2GF; 
      out_prims_r[ww].gf = auxevol_gfs + Nxxp2NG012*VALENCIAV_RU2GF; 
      out_prims_l[ww].gf = auxevol_gfs + Nxxp2NG012*VALENCIAV_LU2GF; 
    ww++;
    in_prims[ww].gf      = auxevol_gfs + Nxxp2NG012*BU0GF; 
      out_prims_r[ww].gf = auxevol_gfs + Nxxp2NG012*B_RU0GF; 
      out_prims_l[ww].gf = auxevol_gfs + Nxxp2NG012*B_LU0GF; 
    ww++;
    in_prims[ww].gf      = auxevol_gfs + Nxxp2NG012*BU1GF; 
      out_prims_r[ww].gf = auxevol_gfs + Nxxp2NG012*B_RU1GF; 
      out_prims_l[ww].gf = auxevol_gfs + Nxxp2NG012*B_LU1GF; 
    ww++;
    in_prims[ww].gf      = auxevol_gfs + Nxxp2NG012*BU2GF; 
      out_prims_r[ww].gf = auxevol_gfs + Nxxp2NG012*B_RU2GF; 
      out_prims_l[ww].gf = auxevol_gfs + Nxxp2NG012*B_LU2GF; 
    ww++;

    // Prims are defined AT ALL GRIDPOINTS, so we set the # of ghostzones to zero:
    for(int i=0;i<NUM_RECONSTRUCT_GFS;i++) for(int j=1;j<=3;j++) { in_prims[i].gz_lo[j]=0; in_prims[i].gz_hi[j]=0; }
    // Left/right variables are not yet defined, yet we set the # of gz's to zero by default:
    for(int i=0;i<NUM_RECONSTRUCT_GFS;i++) for(int j=1;j<=3;j++) { out_prims_r[i].gz_lo[j]=0; out_prims_r[i].gz_hi[j]=0; }
    for(int i=0;i<NUM_RECONSTRUCT_GFS;i++) for(int j=1;j<=3;j++) { out_prims_l[i].gz_lo[j]=0; out_prims_l[i].gz_hi[j]=0; }

    ww=0;
    which_prims_to_reconstruct[ww]=VX; ww++;
    which_prims_to_reconstruct[ww]=VY; ww++;
    which_prims_to_reconstruct[ww]=VZ; ww++;
    which_prims_to_reconstruct[ww]=BX; ww++;
    which_prims_to_reconstruct[ww]=BY; ww++;
    which_prims_to_reconstruct[ww]=BZ; ww++;
    num_prims_to_reconstruct=ww;

    REAL quarter = 0.25;
    
    for(int flux_dirn=0;flux_dirn<3;flux_dirn++) {
        // This function is housed in the file: "reconstruct_set_of_prims_PPM_GRFFE_NRPy.c"
        reconstruct_set_of_prims_PPM_GRFFE_NRPy(&params, auxevol_gfs, flux_dirn, num_prims_to_reconstruct,                                                          
                                                which_prims_to_reconstruct, in_prims, out_prims_r, out_prims_l, temporary);
        if(flux_dirn==0) {
            calculate_E_fluxD0(params,xx,auxevol_gfs);
            LOOP_REGION(0,Nxx_plus_2NGHOSTS2,0,Nxx_plus_2NGHOSTS1,0,Nxx_plus_2NGHOSTS0) {
                rhs_gfs[IDX4S(AD2GF,i0,i1,i2)] -= quarter*(auxevol_gfs[IDX4S(E_FLUXD0GF,i0,i1,i2)] + auxevol_gfs[IDX4S(E_FLUXD0GF,i0+1,i1,i2)]);
                rhs_gfs[IDX4S(AD1GF,i0,i1,i2)] += quarter*(auxevol_gfs[IDX4S(E_FLUXD0GF,i0,i1,i2)] + auxevol_gfs[IDX4S(E_FLUXD0GF,i0+1,i1,i2)]);
            }
        }
        else if(flux_dirn==1) {
            calculate_E_fluxD1(params,xx,auxevol_gfs);
            LOOP_REGION(0,Nxx_plus_2NGHOSTS2,0,Nxx_plus_2NGHOSTS1,0,Nxx_plus_2NGHOSTS0) {
                rhs_gfs[IDX4S(AD0GF,i0,i1,i2)] -= quarter*(auxevol_gfs[IDX4S(E_FLUXD1GF,i0,i1,i2)] + auxevol_gfs[IDX4S(E_FLUXD1GF,i0,i1+1,i2)]);
                rhs_gfs[IDX4S(AD2GF,i0,i1,i2)] += quarter*(auxevol_gfs[IDX4S(E_FLUXD1GF,i0,i1,i2)] + auxevol_gfs[IDX4S(E_FLUXD1GF,i0,i1+1,i2)]);
            }
        }
        else {
            calculate_E_fluxD2(params,xx,auxevol_gfs);
            LOOP_REGION(0,Nxx_plus_2NGHOSTS2,0,Nxx_plus_2NGHOSTS1,0,Nxx_plus_2NGHOSTS0) {
                rhs_gfs[IDX4S(AD1GF,i0,i1,i2)] -= quarter*(auxevol_gfs[IDX4S(E_FLUXD2GF,i0,i1,i2)] + auxevol_gfs[IDX4S(E_FLUXD2GF,i0,i1,i2+1)]);
                rhs_gfs[IDX4S(AD0GF,i0,i1,i2)] += quarter*(auxevol_gfs[IDX4S(E_FLUXD2GF,i0,i1,i2)] + auxevol_gfs[IDX4S(E_FLUXD2GF,i0,i1,i2+1)]);
            }
        }
    }
}


Appending to Validation//A2B_unit_test.c


<a id='compile_run'></a>

## Step 2.a: Compile and run the code

$$\label{compile_run}$$

Now that we have our file, we can compile it and run the executable.

In [12]:
import time

print("Now compiling, should take ~2 seconds...\n")
start = time.time()
cmd.C_compile(os.path.join(out_dir,"Induction_Equation_unit_test.c"), os.path.join(out_dir,"Induction_Equation_unit_test"))
end = time.time()
print("Finished in "+str(end-start)+" seconds.\n\n")

print("Now running...\n")
start = time.time()
!./Validation/Induction_Equation_unit_test
# To do a convergence test, we'll also need a second grid with twice the resolution.
end = time.time()
print("Finished in "+str(end-start)+" seconds.\n\n")


Now compiling, should take ~2 seconds...

Compiling executable...
Executing `gcc -Ofast -fopenmp -march=native -funroll-loops Validation/A2B_unit_test.c -o Validation/A2B_unit_test -lm`...
Finished executing in 0.8151066303253174 seconds.
Finished compilation.
Finished in 0.8266806602478027 seconds.


Now running...

dxx0,dxx1,dxx2 = 3.33333e-03,3.33333e-03,3.33333e-03
x[0] = -1.00000e-02
x[1] = -6.66667e-03
x[2] = -3.33333e-03
x[3] = 4.33681e-19
x[4] = 3.33333e-03
x[5] = 6.66667e-03
x[6] = 1.00000e-02
dxx0,dxx1,dxx2 = 1.66667e-03,1.66667e-03,1.66667e-03
x[0] = -1.00000e-02
x[1] = -8.33333e-03
x[2] = -6.66667e-03
x[3] = -5.00000e-03
x[4] = -3.33333e-03
x[5] = -1.66667e-03
x[6] = 4.33681e-19
x[7] = 1.66667e-03
x[8] = 3.33333e-03
x[9] = 5.00000e-03
x[10] = 6.66667e-03
x[11] = 8.33333e-03
x[12] = 1.00000e-02
dxx0,dxx1,dxx2 = 8.33333e-04,8.33333e-04,8.33333e-04
x[0] = -1.00000e-02
x[1] = -9.16667e-03
x[2] = -8.33333e-03
x[3] = -7.50000e-03
x[4] = -6.66667e-03
x[5] = -5.83333e-03
x[6] = -5.

<a id='latex_pdf_output'></a>

# Step 3: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]

$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Start_to_Finish-GiRaFFE_NRPy-A2B.pdf](Tutorial-Start_to_Finish-GiRaFFE_NRPy-A2B.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [14]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx --log-level='WARN' Tutorial-Start_to_Finish-GiRaFFE_NRPy-A2B.ipynb
!pdflatex -interaction=batchmode Tutorial-Start_to_Finish-GiRaFFE_NRPy-A2B.tex
!pdflatex -interaction=batchmode Tutorial-Start_to_Finish-GiRaFFE_NRPy-A2B.tex
!pdflatex -interaction=batchmode Tutorial-Start_to_Finish-GiRaFFE_NRPy-A2B.tex
!rm -f Tut*.out Tut*.aux Tut*.log

This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
